In [1]:
import numpy as np
import pandas as pd
import plotly
import dash
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc
from dash import dash_table
from random import randint, seed
import datetime
from dash.exceptions import PreventUpdate
import base64
import io

## Control widgets

# Baseline Function (Dynamic Pricing)

In [2]:
constraint_table = pd.read_csv('Constraints.csv')

In [3]:
def dynamic_func(price,data_prop,voice_prop,sms_prop,offnet_prop,table):
    control_factor = 1
    control_factor_data =1
    control_factor_voice=1
    control_factor_sms =1
    peak_factor_data = 1
    peak_factor_voice = 1
    peak_factor_sms = 1
    if(data_prop+voice_prop+sms_prop+offnet_prop != 1):
        data_prop = data_prop/(data_prop+voice_prop+sms_prop+offnet_prop)
        voice_prop = voice_prop/(data_prop+voice_prop+sms_prop+offnet_prop)
        sms_prop = sms_prop/(data_prop+voice_prop+sms_prop+offnet_prop)
        offnet_prop = offnet_prop/(data_prop+voice_prop+sms_prop+offnet_prop)
        
    cols = [x for x in table.columns[3:]]
    condition = (table['Min Range']<=price) & (table['Max Range']>price)
    try:
        ppmb,ppmin,ppmin_o,ppsms,val,data_limit,voice_limit,offnet_limit,sms_limit = list(table[condition][cols].iloc[0])

        resource_mbs = (price*data_prop*peak_factor_data*control_factor_data)/ppmb

        resource_mbs = min(np.ceil(resource_mbs/10)*10,data_limit)
        resource_min = (price*voice_prop*control_factor_voice*peak_factor_voice)/ppmin
        resource_min = min(np.ceil(resource_min/10)*10,voice_limit)
        resource_sms = (price*sms_prop*control_factor_sms*peak_factor_sms)/ppsms
        resource_sms = min(np.ceil(resource_sms/10)*10,sms_limit)
        resource_offnet = price*offnet_prop/ppmin_o
        resource_offnet = min(np.ceil(resource_offnet/10)*10,offnet_limit)
        effective_ppmb = price*data_prop/resource_mbs if resource_mbs > 0 else 0
        effective_ppmin = price*voice_prop/resource_min if resource_min > 0 else 0
        effective_ppsms = price*sms_prop/resource_sms if resource_sms > 0 else 0
        effective_ppmin_offnet = price*offnet_prop/resource_offnet if resource_offnet > 0 else 0

        return [resource_mbs, resource_min, resource_sms,effective_ppmb,effective_ppmin,effective_ppsms,val,resource_offnet,effective_ppmin_offnet]

    except:

        return [0,0,0,0,0,0,0,0,0]
    
    

# Create Initial Output

In [4]:
ppmb_val,ppmin_val,ppsms_val = 1,1,1
d_prop,v_prop,s_prop,o_prop = 0.5,0.3,0.2,0.2
price_list = [100+5*x for x in range(1,int(900/5))]
# price_list
price_list_7 = [100+5*x for x in range(1,int(305/5))]
price_list_30 = [295+5*x for x in range(1,int(900/5))]
dyn_out = [dynamic_func(p,d_prop,v_prop,s_prop,o_prop,constraint_table) for p in price_list]
output_df= pd.DataFrame()
output_df['Price'] = [x for x in price_list]
output_df['Data MBs'] =[x[0] for x in dyn_out]
output_df['Onnet Mins'] =[x[1] for x in dyn_out]
output_df['Offnet Mins'] =[x[7] for x in dyn_out]
output_df['SMS'] =[x[2] for x in dyn_out]
output_df['Validity'] = [x[6] for x in dyn_out]
output_df['PPMB Diluted'] = [x[3] for x in dyn_out]
output_df['PPMIN Diluted'] = [x[4] for x in dyn_out]
output_df['PPSMS Diluted'] = [x[5] for x in dyn_out]
output_df['PPMIN OFFNET Diluted'] = [x[8] for x in dyn_out]

# Sub plots (PPMB, PPMIN, PPSMS)

In [5]:
ppmb_subplot = make_subplots(rows=2, cols=2, shared_xaxes=False, vertical_spacing=0.2)
condition_7 = (output_df['Price']<300)
condition_30 = (output_df['Price']>=300)
ppmb_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Data MBs'],y=output_df[condition_7]['PPMB Diluted'],mode='lines+markers',name ='PPMB Diluted vs Usage (7 Day)',
                                       marker=dict(color='blue',size=1),line = dict(color='blue', width=2,dash='dash')),
                      row=1,col=1
                      )
ppmb_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Price'],y=output_df[condition_7]['PPMB Diluted'],mode='lines+markers',name ='PPMB Diluted vs Price (7 Day)',
                                       marker=dict(color='blue',size=1),line = dict(color='blue', width=2,dash='dash')),
                      row=2,col=1)

ppmb_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Data MBs'],y=output_df[condition_30]['PPMB Diluted'],mode='lines+markers',name ='PPMB Diluted vs Usage (30 Day)',
                                       marker=dict(color='blue',size=1),line = dict(color='blue', width=2,dash='solid')),
                      row=1,col=2
                      )
ppmb_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Price'],y=output_df[condition_30]['PPMB Diluted'],mode='lines+markers',name ='PPMB Diluted vs Price (30 Day)',
                                       marker=dict(color='blue',size=1),line = dict(color='blue', width=2,dash='solid')),
                      row=2,col=2)




ppmb_subplot.update_layout(
    title="PPMB Diluted",
    xaxis1_title="Data MBs",
    yaxis1_title="PPMB",
    
    xaxis3_title="Price",
    yaxis3_title="PPMB",
    
    xaxis2_title="Data MBs",
    yaxis2_title="PPMB",
    
    xaxis4_title="Price",
    yaxis4_title="PPMB",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    )
    
)


ppmb_fig= go.FigureWidget(ppmb_subplot)


ppmin_subplot = make_subplots(rows=2, cols=2, shared_xaxes=False, vertical_spacing=0.2)
ppmin_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Onnet Mins'],y=output_df[condition_7]['PPMIN Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Usage (7 Day)',
                                       marker=dict(color='green',size=1),line = dict(color='green', width=2,dash='dash')),
                      row=1,col=1
                      )
ppmin_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Price'],y=output_df[condition_7]['PPMIN Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Price (7 Day)',
                                       marker=dict(color='green',size=1),line = dict(color='green', width=2,dash='dash')),
                      row=2,col=1)

ppmin_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Onnet Mins'],y=output_df[condition_30]['PPMIN Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Usage (30 Day)',
                                       marker=dict(color='green',size=1),line = dict(color='green', width=2,dash='solid')),
                      row=1,col=2
                      )
ppmin_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Price'],y=output_df[condition_30]['PPMIN Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Price (30 Day)',
                                       marker=dict(color='green',size=1),line = dict(color='green', width=2,dash='solid')),
                      row=2,col=2)




ppmin_subplot.update_layout(
    title="PPMIN Diluted",
    xaxis1_title="Onnet Mins",
    yaxis1_title="PPMIN",
    
    xaxis3_title="Price",
    yaxis3_title="PPMIN",
    
    xaxis2_title="Onnet Mins",
    yaxis2_title="PPMIN",
    
    xaxis4_title="Price",
    yaxis4_title="PPMIN",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    )
    
)


ppmin_fig= go.FigureWidget(ppmin_subplot)


ppsms_subplot = make_subplots(rows=2, cols=2, shared_xaxes=False, vertical_spacing=0.2)
ppsms_subplot.add_trace(go.Scatter(x=output_df[condition_7]['SMS'],y=output_df[condition_7]['PPSMS Diluted'],mode='lines+markers',name ='PPsms Diluted vs Usage (7 Day)',
                                       marker=dict(color='purple',size=1),line = dict(color='purple', width=2,dash='dash')),
                      row=1,col=1
                      )
ppsms_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Price'],y=output_df[condition_7]['PPSMS Diluted'],mode='lines+markers',name ='PPsms Diluted vs Price (7 Day)',
                                       marker=dict(color='purple',size=1),line = dict(color='purple', width=2,dash='dash')),
                      row=2,col=1)

ppsms_subplot.add_trace(go.Scatter(x=output_df[condition_30]['SMS'],y=output_df[condition_30]['PPSMS Diluted'],mode='lines+markers',name ='PPsms Diluted vs Usage (30 Day)',
                                       marker=dict(color='purple',size=1),line = dict(color='purple', width=2,dash='solid')),
                      row=1,col=2
                      )
ppsms_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Price'],y=output_df[condition_30]['PPSMS Diluted'],mode='lines+markers',name ='PPsms Diluted vs Price (30 Day)',
                                       marker=dict(color='purple',size=1),line = dict(color='purple', width=2,dash='solid')),
                      row=2,col=2)




ppsms_subplot.update_layout(
    title="PPSMS Diluted",
    xaxis1_title="SMS",
    yaxis1_title="PPSMS",
    
    xaxis3_title="Price",
    yaxis3_title="PPSMS",
    
    xaxis2_title="SMS",
    yaxis2_title="PPSMS",
    
    xaxis4_title="Price",
    yaxis4_title="PPSMS",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    )
    
)


ppsms_fig= go.FigureWidget(ppsms_subplot)



In [6]:




ppmin_o_subplot = make_subplots(rows=2, cols=2, shared_xaxes=False, vertical_spacing=0.2)
ppmin_o_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Offnet Mins'],y=output_df[condition_7]['PPMIN OFFNET Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Usage (7 Day)',
                                       marker=dict(color='red',size=1),line = dict(color='red', width=2,dash='dash')),
                      row=1,col=1
                      )
ppmin_o_subplot.add_trace(go.Scatter(x=output_df[condition_7]['Price'],y=output_df[condition_7]['PPMIN OFFNET Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Price (7 Day)',
                                       marker=dict(color='red',size=1),line = dict(color='red', width=2,dash='dash')),
                      row=2,col=1)

ppmin_o_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Offnet Mins'],y=output_df[condition_30]['PPMIN OFFNET Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Usage (30 Day)',
                                       marker=dict(color='red',size=1),line = dict(color='red', width=2,dash='solid')),
                      row=1,col=2
                      )
ppmin_o_subplot.add_trace(go.Scatter(x=output_df[condition_30]['Price'],y=output_df[condition_30]['PPMIN OFFNET Diluted'],mode='lines+markers',name ='PPMIN Diluted vs Price (30 Day)',
                                       marker=dict(color='red',size=1),line = dict(color='red', width=2,dash='solid')),
                      row=2,col=2)




ppmin_o_subplot.update_layout(
    title="PPMIN Offnet Diluted",
    xaxis1_title="Offnet Mins",
    yaxis1_title="PPMIN Offnet",
    
    xaxis3_title="Price",
    yaxis3_title="PPMIN Offnet",
    
    xaxis2_title="Offnet Mins",
    yaxis2_title="PPMIN Offnet",
    
    xaxis4_title="Price",
    yaxis4_title="PPMIN Offnet",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    )
    
)


ppmin_o_fig= go.FigureWidget(ppmin_o_subplot)

# Main Plots (Data MBs, Voice Mins, SMS)

In [7]:



data_fig = go.FigureWidget(data=[

    go.Scatter(
        x=price_list_7,y=output_df['Data MBs'],mode='lines+markers',name ='7 Day Validity (Data)',
        marker=dict(color='blue',size=1),line = dict(color='blue', width=2, dash='dash')
    )
    ,
    go.Scatter(
        x=price_list_30,y=output_df['Data MBs'],mode='lines+markers',name ='30 Day Validity (Data)',
        marker=dict(color='blue',size=1),line = dict(color='blue', width=2, dash='solid')
    )
])

voice_fig = go.FigureWidget(data=[

    go.Scatter(
        x=price_list_7,y=output_df['Onnet Mins'],mode='lines+markers',name ='7 Day Validity (Mins)',
        marker=dict(color='green',size=1),line = dict(color='green', width=2, dash='dash')
    ),
    go.Scatter(
        x=price_list_30,y=output_df['Onnet Mins'],mode='lines+markers',name ='30 Day Validity (Mins)',
        marker=dict(color='green',size=1),line = dict(color='green', width=2, dash='solid')
    )
])

offnet_fig = go.FigureWidget(data=[
#     go.Scatter(
#         x=list(output_df['Price']), y=list(output_df['Onnet Mins']),
#         mode='markers',name ='Onnet Mins Baseline', marker= dict(color='green')
#     )
#     ,
    go.Scatter(
        x=price_list_7,y=output_df['Offnet Mins'],mode='lines+markers',name ='7 Day Validity (Mins)',
        marker=dict(color='red',size=1),line = dict(color='red', width=2, dash='dash')
    ),
    go.Scatter(
        x=price_list_30,y=output_df['Offnet Mins'],mode='lines+markers',name ='30 Day Validity (Mins)',
        marker=dict(color='red',size=1),line = dict(color='red', width=2, dash='solid')
    )
])

# sms_fig = go.FigureWidget(data = [
# #     go.Scatter(
# #         x=list(output_df['Price']), y=list(output_df['SMS']),
# #         mode='markers',name='SMS Baseline',marker= dict(color='red')
# #     )
# #     , 
#     go.Scatter(
#         x=price_list_7,y=output_df['SMS'],mode='lines+markers',name ='7 Day Validity (SMS)',
#         marker=dict(color='red',size=1),line = dict(color='red', width=2, dash='dash')
#     ) ,
#     go.Scatter(
#         x=price_list_30,y=output_df['SMS'],mode='lines+markers',name ='30 Day Validity (SMS)',
#         marker=dict(color='red',size=1),line = dict(color='red', width=2, dash='solid')
#     )
    
# ])

data_fig.update_layout(
    title="Data Presented (MBs)",
    xaxis_title="Price",
    yaxis_title="Data MBs",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    )
    ,width =500,height=400
    ,legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    margin=dict(l=10, r=20, t=30, b=0),
)

voice_fig.update_layout(
    title="Voice Presented (Mins)",
    xaxis_title="Price",
    yaxis_title="Mins",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    ),width =500,height=400
    ,legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
    ,margin=dict(l=10, r=20, t=30, b=0),
)

offnet_fig.update_layout(
    title="Offnet Min Presented",
    xaxis_title="Price",
    yaxis_title="Offnet Mins",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    ),width =500,height=400
    ,legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
    ,margin=dict(l=10, r=20, t=30, b=0),
)


ppmb_subplot.update_layout(
    title="PPMB Diluted",
    xaxis1_title="Data MBs ",
    yaxis1_title="PPMB",
    
    xaxis3_title="Price ",
    yaxis3_title="PPMB",
    
    xaxis2_title="Data MBs ",
    yaxis2_title="PPMB",
    
    xaxis4_title="Price ",
    yaxis4_title="PPMB",
    font=dict(
        family="Tahoma",
        size=12,
        color="Black"
    )
    
)




print()


# Indicators 

In [8]:
d_prop_ind = go.Indicator(value=d_prop,title={'text':'Data Proportion','font_size':15},mode='number',number = {'font_size': 15,'valueformat': '.0%'},
                       domain={'row':0,'column':0})
v_prop_ind = go.Indicator(value=v_prop,title={'text':'Voice Proportion','font_size':15},mode='number',number = {'font_size': 15,'valueformat': '.0%'},
                       domain={'row':0,'column':1})
s_prop_ind = go.Indicator(value=s_prop,title={'text':'SMS Proportion','font_size':15},mode='number',number = {'font_size': 15,'valueformat': '.0%'},
                       domain={'row':0,'column':3})
o_prop_ind = go.Indicator(value=o_prop,title={'text':'Offnet Voice Proportion','font_size':15},mode='number',number = {'font_size': 15,'valueformat': '.0%'},
                       domain={'row':0,'column':2})
props_ind = go.Figure()
props_ind.add_trace(d_prop_ind)
props_ind.add_trace(v_prop_ind)
props_ind.add_trace(s_prop_ind)
props_ind.add_trace(o_prop_ind)
props_ind.update_layout(
    grid = {'rows': 1, 'columns': 4, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Speed"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }},height=100,width=1000,margin=dict(l=2, r=2, t=2, b=2))
print()

In [9]:
price = 105
dyn_output = dynamic_func(price,d_prop,v_prop,s_prop,o_prop,constraint_table)


Mbs = go.Indicator(value=dyn_output[0],title={'text':'Data Mbs','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':0})
Min = go.Indicator(value=dyn_output[1],title={'text':'Onnet Mins','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':1})
SMS = go.Indicator(value=dyn_output[2],title={'text':'SMS','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':2})
offnet_ = go.Indicator(value=dyn_output[7],title={'text':'Offnet Mins','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':3})
ppmb_ = go.Indicator(value=dyn_output[3],title={'text':'PPMB','font_size':15},mode='number',number = {'font_size': 15},
                      domain={'row':0,'column':5} )
ppmin_ = go.Indicator(value=dyn_output[4],title={'text':'PPMIN','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':6})
ppsms_ = go.Indicator(value=dyn_output[5],title={'text':'PPSMS','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':7})
ppmin_o = go.Indicator(value=dyn_output[8],title={'text':'PPMIN OFFNET','font_size':15},mode='number',number = {'font_size': 15},
                      domain={'row':0,'column':8} )
Val = go.Indicator(value=dyn_output[6],title={'text':'Validity','font_size':15},mode='number',number = {'font_size': 15},
                       domain={'row':0,'column':4})
inds = go.Figure()
# ppm_ind =go.FigureWidget()
inds.add_trace(Mbs)
inds.add_trace(Min)
inds.add_trace(SMS)
inds.add_trace(offnet_)
inds.add_trace(Val)
inds.add_trace(ppmb_)
inds.add_trace(ppmin_)
inds.add_trace(ppsms_)
inds.add_trace(ppmin_o)


inds.update_layout(
    grid = {'rows': 1, 'columns': 9, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Speed"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }},height=100,width=1100,margin=dict(l=2, r=5, t=2, b=2))
print()


# Comparison Function (Dynamic Pricing)

In [10]:

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output  ,State
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])
# clicks =0

Table = dash_table.DataTable(
        id='Table',
        columns=[{'id': x, 'name': x} for x in constraint_table.columns],
        data= constraint_table.to_dict('records'),
        editable=True,
    fixed_rows={'headers': True},
    style_table={'height': '300px', 'overflowY': 'auto'}
    )
Load =  dcc.Upload(
        id='upload',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select File')
        ]),
        style={
            'width': '100%', 'height': '60px', 'lineHeight': '60px',
            'borderWidth': '1px', 'borderStyle': 'dashed',
            'borderRadius': '5px', 'textAlign': 'center', 'margin': '10px'
        },
    )
button =html.Button('Save Table', id='button', n_clicks=0)
app.layout = html.Div([

    html.H1("Dynamic Simulator - Recharge Upsell (MTA)", style={'text-align': 'center'}),

    html.Div(children=[
        
    html.Label('Data Prop'),
    
    dcc.Slider(0, 1, 0.05,marks=None, tooltip={"placement": "bottom", "always_visible": True},
               value=0.5,
               id='Data Prop',updatemode='drag'
               
    ),
    html.Label('Voice Prop'),
    dcc.Slider(0, 1, 0.05,
               value=0.3,marks=None, tooltip={"placement": "bottom", "always_visible": True},
               id='Voice Prop',updatemode='drag'

    ),
    html.Label('Offnet Prop'),
  dcc.Slider(0, 1, 0.05,
               value=0.2,marks=None, tooltip={"placement": "bottom", "always_visible": True},
               id='Offnet Prop',updatemode='drag'
             
    ),
    html.Label('SMS Prop'),
  dcc.Slider(0, 1, 0.05,
               value=0.2,marks=None, tooltip={"placement": "bottom", "always_visible": True},
               id='SMS Prop',updatemode='drag'
             
    )
    ,
   html.Label('Price'),
  dcc.Slider(105, 1000, 1,
               value=105,marks=None, tooltip={"placement": "bottom", "always_visible": True},
               id='Price',updatemode='drag'
             
    ),
    
    ],style = {'display': 'inline-block', 'width': '50%','vertical-align': 'top', 'margin-left': '25vw', 'margin-top': '0vw'}),

    
    
    html.Div(children=[
     dbc.Row([
         dbc.Col(dcc.Graph(id='Prop Indicators', figure=props_ind),
                    width=12, lg={'size': 12,  "offset": 0, 'order': 'first'}
                        )
         ,
         
         dbc.Col(dcc.Graph(id='Indicators', figure=inds),
                    width=12, lg={'size': 12,  "offset": 0, 'order': 'first'}
                        )])
             
    ],style = {'display': 'inline-block', 'width': '50%','vertical-align': 'top', 'margin-left': '17vw', 'margin-top': '0vw'})
            ,
    
    html.Br(),

    html.H2("Offer Presented Resources - Visualization", style={'text-align': 'center'}),
    html.Br(),
    html.Div(children=[
        dbc.Row([
           dbc.Col(dcc.Graph(id='Data Presented', figure=data_fig),
                        width=4, lg={'size': 4,  "offset": 0, 'order': 'first'}
                        ),
          dbc.Col(dcc.Graph(id='Mins Presented', figure=voice_fig),
                        width=4, lg={'size': 4,  "offset": 0, 'order': 'second'}
                        ),
        dbc.Col(dcc.Graph(id='Offnet Mins Presented', figure=offnet_fig),
                        width=4, lg={'size': 4,  "offset": 0, 'order': 'last'}
                        )
        ]),
]),
    html.Br(),
    html.H2("Price Control Table", style={'text-align': 'center'}),
    dbc.Row([dbc.Col(button,width =1,lg={'size': 1,  "offset": 0, 'order': 'first'}),
             
             dbc.Col(Load,width =10,lg={'size': 10,  "offset": 0, 'order': 'second'})
            
            ]),
    dbc.Row([Table
            
            
            
            ]),
    html.Br(),
    html.H2("Average Price per Usage (Visualization)", style={'text-align': 'center'}),

    dcc.Graph(id='PPMB Diluted',figure=ppmb_fig),
    
    dcc.Graph(id='PPMIN Diluted',figure=ppmin_fig),
    dcc.Graph(id='PPMIN OFFNET Diluted', figure=ppmin_o_fig),
    dcc.Graph(id='PPSMS Diluted', figure=ppsms_fig),
    

])
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    if 'csv' in filename:
        # Assume that the user uploaded a CSV file
        return pd.read_csv(
            io.StringIO(decoded.decode('utf-8')))
    elif 'xls' in filename:
        # Assume that the user uploaded an excel file
        return pd.read_excel(io.BytesIO(decoded))

@app.callback(
    [
#         Output('Table','figure'),
    Output('Data Presented','figure'),
    Output('Mins Presented','figure'),
    Output('Offnet Mins Presented','figure'),
    Output('PPMB Diluted','figure'),
    Output('PPMIN Diluted','figure'),
    Output('PPMIN OFFNET Diluted','figure'),    
    Output('PPSMS Diluted','figure'),
    Output('Indicators','figure'),
    Output('Prop Indicators','figure')
        
        
    ],
    [Input('Data Prop',component_property='value'),
    Input('Voice Prop',component_property='value'),
    Input('Offnet Prop',component_property='value'),
    Input('SMS Prop',component_property='value'),
    
    Input('Price',component_property='value'),
    Input('Table', 'data'),
    Input('Table', 'columns'),

    ]
)  

def update_props(d_prop,v_prop,o_prop,s_prop,slider_p,rows,columns):
#     print(d_prop,v_prop,s_prop,ppmbs,ppmins,ppsmss,baseline,pf,dpf,vpf,spf)
    constraint_table = pd.DataFrame(rows, columns=[c['name'] for c in columns]).astype('float')
    max_price_7 = max(constraint_table[constraint_table['Validity']==7]['Max Range'])
#     display(constraint_table)


    d_p,v_p,s_p,o_p = d_prop,v_prop,s_prop,o_prop
    price = slider_p
    dyn_output = dynamic_func(price,d_prop,v_prop,s_prop,o_prop,constraint_table)
    inds.data[0].value = dyn_output[0]
    inds.data[1].value = dyn_output[1]
    inds.data[2].value = dyn_output[2]
    inds.data[3].value = dyn_output[7]
    inds.data[4].value = dyn_output[6]
    inds.data[5].value = dyn_output[3]
    inds.data[6].value = dyn_output[4]
    inds.data[7].value = dyn_output[5]
    inds.data[8].value = dyn_output[8]
    
    dyn_out = [dynamic_func(p,d_prop,v_prop,s_prop,o_prop,constraint_table) for p in price_list]
        
    output_df= pd.DataFrame()
    output_df['Price'] = [x for x in price_list]
    output_df['Data MBs'] =[x[0] for x in dyn_out]
    output_df['Onnet Mins'] =[x[1] for x in dyn_out]
    output_df['Offnet Mins'] =[x[7] for x in dyn_out]
    output_df['SMS'] =[x[2] for x in dyn_out]
    output_df['Validity'] = [x[6] for x in dyn_out]
    output_df['PPMB Diluted'] = [x[3] for x in dyn_out]
    output_df['PPMIN Diluted'] = [x[4] for x in dyn_out]
    output_df['PPSMS Diluted'] = [x[5] for x in dyn_out]
    output_df['PPMIN OFFNET Diluted'] = [x[8] for x in dyn_out]
    condition_7 = (output_df['Price']<=max_price_7) 
    condition_30 = (output_df['Price']>max_price_7)
#     d_p =d_prop
    if(d_prop+v_prop+s_prop+o_prop!=1):
        d_p = d_prop/(d_prop+v_prop+s_prop+o_prop)
        v_p = v_prop/(d_prop+v_prop+s_prop+o_prop)
        s_p = s_prop/(d_prop+v_prop+s_prop+o_prop)
        o_p = o_prop/(d_prop+v_prop+s_prop+o_prop)
    props_ind.data[0].value = d_p
    props_ind.data[1].value = v_p
    props_ind.data[3].value = o_p
    props_ind.data[2].value = s_p
    

    data_fig.data[0].y = output_df[condition_7]['Data MBs']
    data_fig.data[0].x = output_df[condition_7]['Price']
    data_fig.data[1].y = output_df[condition_30]['Data MBs']
    data_fig.data[1].x = output_df[condition_30]['Price']



    voice_fig.data[0].y = output_df[condition_7]['Onnet Mins']
    voice_fig.data[0].x = output_df[condition_7]['Price']
    voice_fig.data[1].y = output_df[condition_30]['Onnet Mins']
    voice_fig.data[1].x = output_df[condition_30]['Price']
        

    offnet_fig.data[0].y = output_df[condition_7]['Offnet Mins']
    offnet_fig.data[0].x = output_df[condition_7]['Price']
    offnet_fig.data[1].y = output_df[condition_30]['Offnet Mins']
    offnet_fig.data[1].x = output_df[condition_30]['Price']
            
    

    ppmb_fig.data[0].y = output_df[condition_7]['PPMB Diluted']
    ppmb_fig.data[0].x = output_df[condition_7]['Data MBs']
    ppmb_fig.data[1].y = output_df[condition_7]['PPMB Diluted']
    ppmb_fig.data[1].x = output_df[condition_7]['Price']
    ppmb_fig.data[2].y = output_df[condition_30]['PPMB Diluted']
    ppmb_fig.data[2].x = output_df[condition_30]['Data MBs']
    ppmb_fig.data[3].y = output_df[condition_30]['PPMB Diluted']
    ppmb_fig.data[3].x = output_df[condition_30]['Price']


    ppmin_fig.data[0].y = output_df[condition_7]['PPMIN Diluted']
    ppmin_fig.data[0].x = output_df[condition_7]['Onnet Mins']
    ppmin_fig.data[1].y = output_df[condition_7]['PPMIN Diluted']
    ppmin_fig.data[1].x = output_df[condition_7]['Price']
    ppmin_fig.data[2].y = output_df[condition_30]['PPMIN Diluted']
    ppmin_fig.data[2].x = output_df[condition_30]['Onnet Mins']
    ppmin_fig.data[3].y = output_df[condition_30]['PPMIN Diluted']
    ppmin_fig.data[3].x = output_df[condition_30]['Price']


            
#     'PPMIN OFFNET Diluted'

    ppmin_o_fig.data[0].y = output_df[condition_7]['PPMIN OFFNET Diluted']
    ppmin_o_fig.data[0].x = output_df[condition_7]['Offnet Mins']
    ppmin_o_fig.data[1].y = output_df[condition_7]['PPMIN OFFNET Diluted']
    ppmin_o_fig.data[1].x = output_df[condition_7]['Price']
    ppmin_o_fig.data[2].y = output_df[condition_30]['PPMIN OFFNET Diluted']
    ppmin_o_fig.data[2].x = output_df[condition_30]['Offnet Mins']
    ppmin_o_fig.data[3].y = output_df[condition_30]['PPMIN OFFNET Diluted']
    ppmin_o_fig.data[3].x = output_df[condition_30]['Price']
    
            



    ppsms_fig.data[0].y = output_df[condition_7]['PPSMS Diluted']
    ppsms_fig.data[0].x = output_df[condition_7]['SMS']
    ppsms_fig.data[1].y = output_df[condition_7]['PPSMS Diluted']
    ppsms_fig.data[1].x = output_df[condition_7]['Price']
    ppsms_fig.data[2].y = output_df[condition_30]['PPSMS Diluted']
    ppsms_fig.data[2].x = output_df[condition_30]['SMS']
    ppsms_fig.data[3].y = output_df[condition_30]['PPSMS Diluted']
    ppsms_fig.data[3].x = output_df[condition_30]['Price']
    
    
    return data_fig,voice_fig,offnet_fig,ppmb_fig,ppmin_fig,ppmin_o_fig,ppsms_fig,inds,props_ind

@app.callback([Output('button','n_clicks')],[Input('button',component_property='n_clicks')
                                             ,Input('Table', 'data'),Input('Table', 'columns')])
def button(n_click,rows,columns):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if('button' in changed_id):
        constraint_table = pd.DataFrame(rows, columns=[c['name'] for c in columns]).astype('float')
        ct = datetime.datetime.now().strftime('%m-%d-%Y %I %M %p')
        name= 'Settings - '+str(ct)
        constraint_table.to_csv(name+'.csv',index=False)
    return [0]
@app.callback([Output('Table', 'data'),
              Output('Table', 'columns')],
              [Input('upload', 'contents'),
              State('upload', 'filename'),
             Input('Table', 'data'),Input('Table', 'columns')]
             )
def update_output(contents, filename,rows,columns):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    constraint_table = pd.DataFrame(rows, columns=[c['name'] for c in columns]).astype('float')
    
    if('upload' in changed_id):
        if contents is None:
            return [constraint_table.to_dict('records'), [{"name": i, "id": i} for i in constraint_table.columns]]

        df = parse_contents(contents, filename)
        if(len(constraint_table.columns)!=len(df.columns)):
            return [constraint_table.to_dict('records'), [{"name": i, "id": i} for i in constraint_table.columns]]
        constraint_table = df

        return [constraint_table.to_dict('records'), [{"name": i, "id": i} for i in constraint_table.columns]]
    return rows,columns
#     raise PreventUpdate

    

app.run_server( mode='external',port = 10001, debug=True)


Dash app running on http://127.0.0.1:10001/


In [11]:
# type(constraint_table.to_dict())
# # , [{"name": i, "id": i} for i in constraint_table.columns]

In [12]:
# [{"name": i, "id": i} for i in constraint_table.columns]